# Inleiding tot Kansrekening en Statistiek
In dit notitieboek gaan we experimenteren met enkele van de concepten die we eerder hebben besproken. Veel concepten uit de kansrekening en statistiek zijn goed vertegenwoordigd in belangrijke bibliotheken voor gegevensverwerking in Python, zoals `numpy` en `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Willekeurige variabelen en verdelingen
Laten we beginnen met het trekken van een steekproef van 30 waarden uit een uniforme verdeling van 0 tot 9. We zullen ook het gemiddelde en de variantie berekenen.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Om visueel in te schatten hoeveel verschillende waarden er in de steekproef zijn, kunnen we de **histogram** plotten:


In [ ]:
plt.hist(sample)
plt.show()

## Analyseren van echte gegevens

Gemiddelde en variantie zijn zeer belangrijk bij het analyseren van gegevens uit de echte wereld. Laten we de gegevens over honkbalspelers laden van [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> We gebruiken hier een package genaamd [**Pandas**](https://pandas.pydata.org/) voor data-analyse. We zullen later in deze cursus meer praten over Pandas en werken met data in Python.

Laten we gemiddelde waarden berekenen voor leeftijd, lengte en gewicht:


In [ ]:
df[['Age','Height','Weight']].mean()

Laten we ons nu richten op de lengte, en standaarddeviatie en variantie berekenen:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Naast het gemiddelde is het zinvol om ook naar de mediaan en kwartielen te kijken. Deze kunnen worden weergegeven met een **boxplot**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

We kunnen ook doosdiagrammen maken van subsets van onze dataset, bijvoorbeeld gegroepeerd op spelersrol.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Opmerking**: Dit diagram suggereert dat gemiddeld de lengtes van eerst base-spelers hoger zijn dan de lengtes van tweede base-spelers. Later zullen we leren hoe we deze hypothese formeler kunnen testen en hoe we kunnen aantonen dat onze gegevens statistisch significant zijn om dit te laten zien.  

Leeftijd, lengte en gewicht zijn allemaal continue stochastische variabelen. Wat denk je dat hun verdeling is? Een goede manier om dat te achterhalen is het plotten van het histogram van de waarden: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normale Verdeling

Laten we een kunstmatige steekproef van gewichten maken die een normale verdeling volgt met dezelfde gemiddelde en variantie als onze echte gegevens:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Aangezien de meeste waarden in het echte leven normaal verdeeld zijn, zouden we geen uniforme willekeurige getallengenerator moeten gebruiken om steekproefgegevens te genereren. Dit is wat er gebeurt als we proberen gewichten te genereren met een uniforme verdeling (gegenereerd door `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Betrouwbaarheidsintervallen

Laten we nu betrouwbaarheidsintervallen berekenen voor het gewicht en de lengte van honkbalspelers. We zullen de code gebruiken [uit deze stackoverflow-discussie](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Hypothesetoetsing

Laten we verschillende posities in onze honkbalspelersdataset verkennen:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Laten we de hypothese testen dat eerste honkmannen langer zijn dan tweede honkmannen. De eenvoudigste manier om dit te doen is door de betrouwbaarheidsintervallen te testen:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

We kunnen zien dat de intervallen niet overlappen.

Een statistisch correctere manier om de hypothese te bewijzen is het gebruik van een **Student t-toets**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

De twee waarden die worden geretourneerd door de `ttest_ind` functie zijn:
* p-waarde kan worden beschouwd als de waarschijnlijkheid dat twee verdelingen dezelfde gemiddelde waarde hebben. In ons geval is deze zeer laag, wat betekent dat er sterk bewijs is dat eerste honkverdedigers langer zijn.
* t-waarde is de tussenliggende waarde van genormaliseerd verschil tussen gemiddelden die wordt gebruikt in de t-toets, en deze wordt vergeleken met een drempelwaarde voor een gegeven betrouwbaarheidswaarde.


## Een normale verdeling simuleren met de centrale limietstelling

De pseudowillekeurige generator in Python is ontworpen om ons een uniforme verdeling te geven. Als we een generator voor een normale verdeling willen maken, kunnen we de centrale limietstelling gebruiken. Om een normaal verdeelde waarde te krijgen, berekenen we gewoon het gemiddelde van een uniform gegenereerde steekproef.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Correlatie en Evil Baseball Corp

Correlatie stelt ons in staat relaties tussen datareeksen te vinden. In ons voorbeeld met speelgoed doen we alsof er een boosaardige honkbalmaatschappij is die haar spelers betaalt op basis van hun lengte - hoe langer de speler is, hoe meer geld hij/zij krijgt. Stel dat er een basissalaris van $1000 is, en een extra bonus van $0 tot $100, afhankelijk van de lengte. We nemen de echte spelers van de MLB en berekenen hun denkbeeldige salarissen:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Laten we nu de covariantie en correlatie van die reeksen berekenen. `np.cov` geeft ons een zogenaamde **covariantiematrix**, wat een uitbreiding is van covariantie naar meerdere variabelen. Het element $M_{ij}$ van de covariantiematrix $M$ is een correlatie tussen invoervariabelen $X_i$ en $X_j$, en diagonale waarden $M_{ii}$ zijn de variantie van $X_{i}$. Op dezelfde manier geeft `np.corrcoef` ons de **correlatiematrix**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Een correlatie gelijk aan 1 betekent dat er een sterke **lineaire relatie** is tussen twee variabelen. We kunnen de lineaire relatie visueel zien door de ene waarde uit te zetten tegen de andere:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Laten we eens kijken wat er gebeurt als de relatie niet lineair is. Stel dat ons bedrijf heeft besloten de voor de hand liggende lineaire afhankelijkheid tussen lengtes en salarissen te verbergen, en wat niet-lineariteit in de formule heeft geïntroduceerd, zoals `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In dit geval is de correlatie iets kleiner, maar nog steeds behoorlijk hoog. Nu, om de relatie nog minder duidelijk te maken, willen we misschien wat extra willekeur toevoegen door een willekeurige variabele aan het salaris toe te voegen. Laten we eens kijken wat er gebeurt:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Kun je raden waarom de stippen zo recht onder elkaar komen te staan in verticale lijnen?

We hebben de correlatie waargenomen tussen een kunstmatig geconstrueerd concept zoals salaris en de waargenomen variabele *lengte*. Laten we ook kijken of de twee waargenomen variabelen, zoals lengte en gewicht, ook correleren:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Helaas hebben we geen resultaten gekregen - alleen enkele vreemde `nan` waarden. Dit komt doordat sommige van de waarden in onze reeks niet gedefinieerd zijn, weergegeven als `nan`, wat ertoe leidt dat het resultaat van de bewerking ook niet gedefinieerd is. Door naar de matrix te kijken, zien we dat `Weight` de problematische kolom is, omdat zelfcorrelatie tussen `Height` waarden is berekend.

> Dit voorbeeld toont het belang aan van **gegevensvoorbereiding** en **schoonmaak**. Zonder juiste gegevens kunnen we niets berekenen.

Laten we de methode `fillna` gebruiken om de ontbrekende waarden in te vullen, en de correlatie berekenen: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Er is inderdaad een correlatie, maar niet zo sterk als in ons kunstmatige voorbeeld. Inderdaad, als we naar de spreidingsdiagram van de ene waarde tegen de andere kijken, zou de relatie veel minder duidelijk zijn:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Conclusie

In dit notitieboek hebben we geleerd hoe we basisbewerkingen op gegevens kunnen uitvoeren om statistische functies te berekenen. We weten nu hoe we een degelijk apparaat van wiskunde en statistiek kunnen gebruiken om enkele hypotheses te bewijzen, en hoe we betrouwbaarheidsintervallen voor willekeurige variabelen kunnen berekenen op basis van een gegevensmonster.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u er rekening mee te houden dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het oorspronkelijke document in de oorspronkelijke taal moet als de gezaghebbende bron worden beschouwd. Voor belangrijke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerd geïnterpreteerde informatie die voortvloeit uit het gebruik van deze vertaling.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
